In [1]:
import os
os.chdir('..')

## Section 6.2

Listing 6.1

In [2]:
from sim_circuit import QuantumRegister, QuantumCircuit
from util import all_close

def geometric_sequence_circuit(n, theta):

    N = 2**n

    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n): # <1>
        qc.h(q[j])

    for j in range(n): # <1>
        qc.p(2 ** j * theta, q[j])

    assert all_close(qc.run(), [1/sqrt(N)*cis(k*theta) for k in range(N)]) # <2>

### Section 6.2.1

Listing 6.2

In [3]:
from math import pi

def encode_frequency(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])

    for j in range(n):
        qc.p(2 * pi / 2 ** (n - j) * v, q[j]) # <1>

    qc.report('geometric_sequence')

    qc.iqft(range(n))

    qc.report('iqft')

    return qc

### Section 6.2.2

In [4]:
n = 3
v = 4.3
qc = encode_frequency(n, v)
state = qc.run()

In [5]:
def prod(iterable):
    p = 1
    for n in iterable:
        p *= n
    return p

In [6]:
from math import cos

N = 2**n
assert all_close([abs(state[k]) for k in range(N)], [
    abs(prod(cos((v - k) * pi / 2 ** (m + 1)) for m in range(n))) for k in
    range(N)])

In [7]:
from util import cis

def complex_sincd(n, v):
    N = 2 ** n
    return [prod(
        cos((v - k) * pi / 2 ** (j + 1)) * cis((v - k) * pi / 2 ** (j + 1))
        for j in range(n)) for k in range(2 ** n)]

In [8]:
assert all_close(state, complex_sincd(3, 4.3))

In [9]:
def complex_sincd_combined_cis(n, v):
    N = 2 ** n
    return [prod(cos((v - k) * pi / 2 ** (m + 1)) for m in range(n)) * cis(
        (N - 1) / N * (v - k) * pi) for k in range(2 ** n)]

In [10]:
assert all_close(state, complex_sincd_combined_cis(3, 4.3))

### Section 6.2.3

Listing 6.3

In [11]:
def geom_alt(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])

    for j in range(n):
        qc.p(pi * 2 ** -j * v, q[j]) # <1>

    return qc

Listing 6.4

In [12]:
def encode_frequency_q_alt(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])
        qc.p(pi * 2 ** -j * v, q[j])

    qc.report('signal')

    qc.iqft(range(n)[::-1], swap=False)

    qc.report('iqft')

    return qc

## Section 6.3

In [13]:
from math import sin, cos
from util import is_close

def discrete_sinc_by_digit(n , v):

    probs = [_ for _ in range(2**n)]
    for l in range(2**n): # <1>
        s = bin(l)[2:].zfill(n)
        assert(len(s) == n)
        p = 1
        k = 0
        for m in range(n): # <2>
            if s[m] == '0':
                p *= cos((v - k)*pi/2**(m+1))**2
            else:
                p *= sin((v - k)*pi/2**(m+1))**2
                k += 2**m

        probs[k] = p
        
    return probs

In [14]:
n = 3
v = 4.7

probs = discrete_sinc_by_digit(n , v)
for k in range(len(probs)):
    assert is_close(probs[k], prod(cos((v-k)*pi/2**(j+1)) for j in range(n))**2)

In [15]:
def recursive_discrete_sinc(n, v):
    if n == 1:
        return [cos(v*pi/2)**2, sin(v*pi/2)**2]

    p = recursive_discrete_sinc(n-1, v)

    return [p[k] * cos((v - k) * pi / 2 ** n) ** 2 for k in
            range(2 ** (n - 1))] + [p[k] * sin((v - k) * pi / 2 ** n) ** 2
                                    for k in range(2 ** (n - 1))]

In [16]:
def discrete_sinc_coin_flips(n, v, count=10000):
    samples = []
    for _ in range(count):
        k = 0
        for m in range(n):
            flip = np.random.binomial(1, sin((v-k)*pi/2**(m+1))**2)
            k += flip*2**m

        samples.append(k)

    return samples

## Section 6.4

### Section 6.4.1

Listing 6.5

In [17]:
def raised_cosine(n, mu):
    N = 2 ** n
    assert (0 <= mu < 2 ** n)

    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.h(q[n - 1])
    qc.p(-pi * mu / N * 2, q[n - 1])

    qc.report('fourier_coefficients')

    qc.qft(q[::-1], False) # <1>

    qc.report('qft')

    return qc

In [18]:
qc = raised_cosine(3, 3.25)
state = qc.run()

In [19]:
from util import print_state_table

print_state_table(state)



Outcome   Binary        Amplitude        Magnitude   Direction      Amplitude Bar         Probability
----------------------------------------------------------------------------------------------------
  0        000     +0.0421 - 0.1389*i     0.1451     -73.14        ##                       0.0211
  1        001     +0.2012 - 0.2452*i     0.3172     -50.63        ######                   0.1006
  2        010     +0.3889 - 0.2079*i     0.4410     -28.13        ########                 0.1945
  3        011     +0.4952 - 0.0488*i     0.4976     -5.63         #########                0.2476
  4        100     +0.4579 + 0.1389*i     0.4785     16.87         #########                0.229
  5        101     +0.2988 + 0.2452*i     0.3865     39.37         #######                  0.1494
  6        110     +0.1111 + 0.2079*i     0.2357     61.88         ####                     0.0556
  7        111     +0.0048 + 0.0488*i     0.0490     84.38                                  0.0024


In [20]:
from math import sqrt, cos
from util import cis

N = 8
mu = 3.25
a = [sqrt(2/N) * cos((k - mu)*pi/N) * cis((k-mu)*pi/N) for k in range(N)]
assert all_close(state, a)

In [21]:
s = N / 2
p = [1 / (2 * s) * (1 + cos((x - mu) / s * pi)) for x in range(N)]
p1 = [1 / s * cos((x - mu) / (2 * s) * pi) ** 2 for x in range(N)]

probs = [2/N*(cos((k - mu)*pi/N))**2 for k in range(N)] 

assert all_close(p, probs)
assert all_close(p1, probs)

### Section 6.4.2

Listing 6.6

In [22]:
from math import acos

def sin_4(n):
    theta = acos(sqrt(2 / 3))
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.ry(2 * theta, q[n - 1])
    qc.p(pi, q[n - 1])
    qc.cry(pi / 2, q[n - 1], q[0])

    for i in range(1, n - 1):
        qc.cx(q[0], q[i])

    qc.report('frequencies')

    qc.qft(q[::-1], False)

    qc.report('qft')

    return qc

In [23]:
n = 3           
N = 2 ** n      
qc = sin_4(n)   
state = qc.run()

In [24]:
print_state_table(state)



Outcome   Binary        Amplitude        Magnitude   Direction      Amplitude Bar         Probability
----------------------------------------------------------------------------------------------------
  0        000     +0.0000 + 0.0000*i     0.0000                                            0.0
  1        001     +0.0846 + 0.0000*i     0.0846     0.0           #                        0.0072
  2        010     +0.2887 + 0.0000*i     0.2887     0.0           #####                    0.0833
  3        011     +0.4928 + 0.0000*i     0.4928     0.0           #########                0.2429
  4        100     +0.5774 + 0.0000*i     0.5774     0.0           ###########              0.3334
  5        101     +0.4928 + 0.0000*i     0.4928     0.0           #########                0.2429
  6        110     +0.2887 + 0.0000*i     0.2887     0.0           #####                    0.0833
  7        111     +0.0846 + 0.0000*i     0.0846     0.0           #                        0.0072


In [25]:
from math import sin

s = [sqrt(8 / (3 * N)) * (sin(k * pi / N)) ** 2 for k in range(N)]
assert all_close(state, s)
p = [8 / 3 / N * (sin(k * pi / N)) ** 4 for k in range(N)] # <1>
assert all_close([abs(state[k])**2 for k in range(N)], p)